# Bird Species Prediction

In this project, I created a convolutional Neural Network which will be able to predict species of the bird. I used different layers and other hyperparameters for building, training and testing this multiclass classifictaion model. I used keras for this project.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

First I mounted my google drive on colab so that I can use the dataset directly from our drive. For this I first uploaded the data on your drive and then mounted the drive on colab.

In [ ]:
# After executing the cell above, Drive files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

After mounting my drive I will locate the folder where my data is stored to use it in our colab notebook. Here I can see that all the folders I have in my drive and 'Bird Species Dataset' contains the images that I will work on.

In [ ]:
# Library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
from os import listdir
from sklearn.preprocessing import  LabelBinarizer
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split

I started by importing some required libraries.

In [ ]:
# Plotting 12 images to check dataset
plt.figure(figsize=(12,12))
path = "/content/drive/My Drive/Bird Speciees Dataset/AMERICAN GOLDFINCH"
for i in range(1,17):
    plt.subplot(4,4,i)
    plt.tight_layout()
    rand_img = imread(path +'/'+ random.choice(sorted(listdir(path))))
    plt.imshow(rand_img)
    plt.xlabel(rand_img.shape[1], fontsize = 10)
    plt.ylabel(rand_img.shape[0], fontsize = 10)

Let's visualize some of the bird images that I will be working on. Also I will observe x and y dimensions of the image.

In [ ]:
# Setting path and creating empty list
dir = "/content/drive/My Drive/Bird Speciees Dataset"
root_dir = listdir(dir)
image_list, label_list = [], []

Setting the root directory for the dataset and storing all the floders name of the dataset. We will also create 2 empty list for image and lables.

In [ ]:
# Reading and converting image to numpy array
for directory in root_dir:
  for files in listdir(f"{dir}/{directory}"):
    image_path = f"{dir}/{directory}/{files}"
    image = cv2.imread(image_path)
    image = img_to_array(image)
    image_list.append(image)
    label_list.append(directory)

Next we will read all the images and convert it into array and appending the list created above with the image and its label.

In [ ]:
# Visualize the number of classes count
label_counts = pd.DataFrame(label_list).value_counts()
label_counts

Check for class imbalance.

In [ ]:
# Storing number of classes
num_classes = len(label_counts)
num_classes

Next I found out the number of classes that I will have to work upon.

In [ ]:
# Checking input image shape
image_list[0].shape

Checking the size of the single image.

In [ ]:
# Checking labels shape 
label_list = np.array(label_list)
label_list.shape

Checking the shape of image labels which will be equal to the number of images I am going to work with.

In [ ]:
# Splitting dataset
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state = 10) 

Now we will split the data into training and testing using train_test_split() of sklearn.

In [ ]:
# Normalize and reshape data
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 224,224,3)
x_test = x_test.reshape( -1, 224,224,3)

Next I normalized the images by dividing them with 255. Also, I reshaped x_train and x_test.

In [ ]:
# Label binarizing
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)
print(lb.classes_)

Next I used label binarizer to one hot encode my data. I also printed the sequence of the classes. 

In [ ]:
# Splitting the training data set into training and validation data sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

Now we will split the training data to validation and training data.

In [ ]:
# Building model architecture
model = Sequential()
model.add(Conv2D(8, (3, 3), padding="same",input_shape=(224,224,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))
model.summary()

Next I created a network architecture for the model. I have used different types of layers according to their features namely Conv_2d (It is used to create a convolutional kernel that is convolved with the input layer to produce the output tensor), max_pooling2d (It is a downsampling technique which takes out the maximum value over the window defined by poolsize), flatten (It flattens the input and creates a 1D output), Dense (Dense layer produce the output as the dot product of input and kernel). In the last layer I used softmax as the activation function because it is a multi class classification problem.

In [ ]:
# Compiling model
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0005),metrics=['accuracy'])

While compiling the model I need to set the type of loss which will be Categorical Crossentropy for my model alongwith this we also need to set the optimizer and the metrics respectively.

In [ ]:
# Training the model
epochs = 50
batch_size = 128
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val))

Fitting the model with the data and finding out the accuracy at each epoch to see how our model is learning. Now I trained my model on 50 epochs and a batch size of 128. I can try using more number of epochs to increase accuracy but here I can see that the model has already reached a very high accuracy so I don't need to run it for more. During each epochs I can see how the model is performing by viewing the training and validation accuracy.

In [ ]:
# Saving model
model.save("/content/drive/My Drive/bird_species.h5")

Now I will save the model in h5 format to use it later for prediction.

In [ ]:
#Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

Next I will plot the accuracy of the model for the training history.

In [ ]:
#Plot the loss history
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

Here we will plot the loss of the model for the training history.

In [ ]:
# Calculating test accuracy
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

Evaluating the model to know the accuracy of the model on the test data.

In [ ]:
# Storing predictions
y_pred = model.predict(x_test)

Here we are storing prediction on test data.

In [ ]:
# Plotting image to compare
img = array_to_img(x_test[5])
img

Visualizing one of the image which we will be further used for prediction.

In [ ]:
# Finding max value from predition list and comaparing original value vs predicted
labels = lb.classes_
print(labels)
print("Originally : ",labels[np.argmax(y_test[5])])
print("Predicted : ",labels[np.argmax(y_pred[5])])

Now, I created list of labels using object of label binarizer. I will print that list and finally I will print out the prediction and the original label of the image we visualized above using argmax(). 

# Conclusion:

I started with loading the dataset into google colab using google drive and visualizing the images. Normalizing is an important step when working with any type of dataset. After that I created a CNN Model which is further used for predicting the bird species using the image supplied to model. 

# Scope:

This project can be used for educational purposes to get a better understanding of how to create network architecture for a CNN model. We can further hyper parameter tune this model to reach a higher accuracy. It can be used by bird sanctuaries to identify different types of birds. 